In [1]:
from __future__ import print_function
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
import random
import keras
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
import tensorflow.keras.backend as K
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

Using TensorFlow backend.


# prepare data

In [2]:
seq_size = 300

In [3]:
def data_prep(path, label, seq_length):
    data_list = []
    for record in SeqIO.parse(path, "fasta"):
        info_tag = record.id
        chromID = info_tag.split("_")[0]
        chromSeq = (str(record.seq)).upper()
        if len(chromSeq) < seq_length:
            continue
        offset_start = int((len(chromSeq)-seq_length)/2.0)
        chromSeq_trimmed = chromSeq[offset_start : offset_start+seq_length]
        data_list.append((chromSeq_trimmed, label, chromID, info_tag))
    return data_list

In [4]:
def create_dataset(dataset, test):
    for data_point in dataset:
        chromID = data_point[2]
        if chromID == "chr9":
            test.append(data_point)

def dataset2onehot(dataset, shuffle=True):
    nucleotides = ["A", "T", "C", "G"]
    def seq2onehot(seq):
        onehot_list = []
        for nuc in seq:
            if nuc == "N":
                onehot = [0.25 for _ in range(len(nucleotides))]
                onehot_list.append(onehot)
            else:
                onehot = [0 for _ in range(len(nucleotides))]
                onehot[nucleotides.index(nuc)] = 1
                onehot_list.append(onehot)
        return onehot_list
    
    def rc(seq):
        return str((Seq(seq)).reverse_complement())
    
    onehot_dataset = []
    for (seq, label, chromID, tag_info) in dataset:
        onehot_dataset.append((seq2onehot(seq), label, (tag_info, "+")))
        onehot_dataset.append((seq2onehot(rc(seq)), label, (tag_info, "-")))
    
    if shuffle:
        random.shuffle(onehot_dataset)
    
    x_list, y_list, info_list = [], [], [] 
    for (x, y, info) in onehot_dataset:
        x_list.append(x)
        y_list.append(y)
        info_list.append(info)
    return np.array(x_list), np.array(y_list), info_list

In [5]:
experiment_path = {"IL4": ["./data/model1_IL4Enhancer/C57Bl6_IL4Enhancers.positive.fa",
                           "./data/model1_IL4Enhancer/C57Bl6_IL4Enhancers.negative.fa",
                           "../../new_project/tmp/IL4_modelJson_tmp.json", 
                           "../../new_project/tmp/IL4_modelWeights.h5"],
                  "IL4Induced": ["./data/model2_IL4InducedEnhancer/Strains_IL4InducedEnhancers.positive.fa",
                                 "./data/model2_IL4InducedEnhancer/Strains_IL4InducedEnhancers.negative.fa",
                                 "../../new_project/tmp/IL4Induced_modelJson_tmp.json", 
                                 "../../new_project/tmp/IL4Induced_modelWeights.h5"],
                  "Induced_vs_noninduced": ["./data/model3_IL4induced_vs_noninduced/Strains_IL4InducedEnhancers.vs.C57_basalNoninducedEnhancers.positive.fa",
                                            "./data/model3_IL4induced_vs_noninduced/Strains_IL4InducedEnhancers.vs.C57_basalNoninducedEnhancers.negative.fa",
                                            "../../new_project/tmp/IL4Induced_vs_noninduced_modelJson_tmp.json", 
                                            "../../new_project/tmp/IL4Induced_vs_noninduced_modelWeights.h5"]}

for experiment_name in experiment_path:
    pos_data_path, neg_data_path, _, _ = experiment_path[experiment_name]
    pos_data = data_prep(pos_data_path, 1, seq_size)
    neg_data = data_prep(neg_data_path, 0, seq_size)

    test_raw = []
    create_dataset(pos_data, test_raw)
    create_dataset(neg_data, test_raw)
    x_test, y_test, info_test = dataset2onehot(test_raw)
    
    for experiment_name_for_model in experiment_path:
        _, _, model_json, model_weights = experiment_path[experiment_name_for_model]
        model = model_from_json(open(model_json).read())
        model.load_weights(model_weights)
        y_pred = model.predict(x_test, batch_size=512, verbose=0)
        roc_value = roc_auc_score(y_test, y_pred)
        pr_value = average_precision_score(y_test, y_pred)
        print ("Data: %s Model: %s auROC: %f auPRC: %f" %(experiment_name, experiment_name_for_model, roc_value, pr_value)) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Data: IL4 Model: IL4 auROC: 0.893664 auPRC: 0.900985
Data: IL4 Model: IL4Induced auROC: 0.840716 auPRC: 0.845858
Data: IL4 Model: Induced_vs_noninduced auROC: 0.381604 auPRC: 0.470984
Data: IL4Induced Model: IL4 auROC: 0.915187 auPRC: 0.909545
Data: IL4Induced Model: IL4Induced auROC: 0.919305 auPRC: 0.921631
Data: IL4Induced Model: Induced_vs_noninduced auROC: 0.560938 auPRC: 0.610653
Data: Induced_vs_noninduced Model: IL4 auROC: 0.458268 auPRC: 0.313658
Data: Induced_vs_noninduced Model: IL4Induced auROC: 0.623467 auPRC: 0.434205
Data: Induced_vs_noninduced Model: Induced_vs_noninduced auROC: 0.796493 auPRC: 0.672665
